In [0]:
%pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/253.9 MB ? eta -:--:--
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/253.9 MB 96.9 MB/s eta 0:00:03
   ━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/253.9 MB 121.2 MB/s eta 0:00:02
   ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/253.9 MB 100.7 MB/s eta 0:00:02
   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/253.9 MB 109.0 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 110.9/253.9 MB 113.2 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 125.6/253.9 MB 106.9 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 138.7/253.9 MB 101.0 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━ 152.0/253.9 MB 91.1 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━ 162.0/253.9 MB 87.4 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 175.1/253.9 MB 83.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 181.1/253.9 MB 79.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━

In [0]:
from pyspark.sql.functions import to_timestamp

 # Load & Preview Data
train_df = spark.read.csv('/FileStore/tables/train.csv/train.csv', header=True, inferSchema=True)
test_df  = spark.read.csv('/FileStore/tables/test.csv/test.csv', header=True, inferSchema=True)

# to_timestamp definition
train_df = train_df.withColumn('Date', to_timestamp('Date'))
test_df  = test_df.withColumn('Date', to_timestamp('Date'))

display(train_df.limit(5))
train_df.printSchema()


Date,store,product,number_sold
2010-01-01T00:00:00.000Z,0,0,801
2010-01-02T00:00:00.000Z,0,0,810
2010-01-03T00:00:00.000Z,0,0,818
2010-01-04T00:00:00.000Z,0,0,796
2010-01-05T00:00:00.000Z,0,0,808


root
 |-- Date: timestamp (nullable = true)
 |-- store: integer (nullable = true)
 |-- product: integer (nullable = true)
 |-- number_sold: integer (nullable = true)



In [0]:
from pyspark.sql.functions import dayofweek, month, year, dayofyear, weekofyear, dayofmonth, last_day

feature_cols = ["store", "product", "dayOfWeek", "month", "year", "dayOfYear", "weekOfYear", "is_month_start", "is_month_end"]

def prep(df):
    return (df
        .withColumn('dayOfWeek', dayofweek('Date'))
        .withColumn('month', month('Date'))
        .withColumn('year', year('Date'))
        .withColumn('dayOfYear', dayofyear('Date'))
        .withColumn('weekOfYear', weekofyear('Date'))
        .withColumn('is_month_start', (dayofmonth('Date') == 1).cast('integer'))
        .withColumn('is_month_end', (dayofmonth('Date') == dayofmonth(last_day('Date'))).cast('integer'))
    )

train_prep = prep(train_df)
test_prep  = prep(test_df)


In [0]:
X_train = train_prep.select(feature_cols).toPandas()
y_train = train_prep.select('number_sold').toPandas().values.ravel()
X_test  = test_prep.select(feature_cols).toPandas()


In [0]:
from xgboost import XGBRegressor

# Use best params found from tuning
best_params = {'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.05}
model = XGBRegressor(**best_params, random_state=42, verbosity=0)
model.fit(X_train, y_train)

# Predict on test
test_preds = model.predict(X_test)

# Attach predictions to test set (as Pandas)
test_out = test_prep.select('Date','store','product','number_sold').toPandas()
test_out['predicted_sales'] = test_preds


In [0]:
print(test_out.head())



        Date  store  product  number_sold  predicted_sales
0 2019-01-01      0        0          845       804.063843
1 2019-01-02      0        0          851       804.063843
2 2019-01-03      0        0          840       804.063843
3 2019-01-04      0        0          842       804.063843
4 2019-01-05      0        0          845       804.063843


In [0]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming you have: test_out['number_sold'] and test_out['predicted_sales']

mae = mean_absolute_error(test_out['number_sold'], test_out['predicted_sales'])
rmse = mean_squared_error(test_out['number_sold'], test_out['predicted_sales'], squared=False)
r2 = r2_score(test_out['number_sold'], test_out['predicted_sales'])
mape = ((abs(test_out['number_sold'] - test_out['predicted_sales']) / test_out['number_sold']).replace([np.inf, -np.inf], np.nan)).mean() * 100

print(f"Test MAE: {mae:.2f}")
print(f"Test RMSE: {rmse:.2f}")
print(f"Test R2: {r2:.3f}")
print(f"Test MAPE: {mape:.2f}%")


Test MAE: 45.42
Test RMSE: 60.80
Test R2: 0.918
Test MAPE: 6.39%
